## Import dependancies and set environment determinism

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

    


2024-04-24 10:03:50.787785: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


378


In [2]:
print(tf.__version__)
print(np.__version__)

2.12.0
1.23.5


In [3]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm
print(keras.__version__)

2.12.0


In [4]:
# if os.path.isdir(res_dir) == False:

def run_model(model_path,l1,l2,loss = 'mae'):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )

    model_history = model.fit(
        trfeat,
        trlabs,
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

In [5]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

In [6]:
home_path = '/home/simonhans/glacierml'
[
        data_path, RGI_path, glathida_path, 
        coregistration_testing_path, 
        arch_test_path, LOO_path
] = gl.set_paths(home_path)

In [7]:
for i in range(1,5,1):
#     l1 = 8
#     l2 = 5
    if i == 1:
        l1 = 10
        l2 = 5
        
    if i == 2:
        l1 = 8
        l2 = 2
        
    if i == 3:
        l1 = 9
        l2 = 5
        
    if i == 4:
        l1 = 6
        l2 = 2
    set_global_determinism(seed=SEED)
    # df = gl.coregister_data(str(i),pth = '/home/simonhans/glacierml/data')
    df = gl.coregister_data_reform(data_path,str(i))
    df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)
    # df = df.sample(frac = 1,random_state = 0)
    df = df.reset_index().drop('index', axis = 1)
    # df = df[list(df)[:-7]]
    trfeat, tefeat, trlabs, telabs = gl.split_data(df)
    print(trfeat)
    

    model = {}
    model_history = {}
    normalizer = {}
    model_path = os.path.join(
        coregistration_testing_path, str(i)
    )
    model = run_model(model_path,l1,l2)
    
    rgi_est_pth = os.path.join(model_path, 'rgi_est_raw.pkl')

    if os.path.isdir(rgi_est_pth) == False:

        RGI = gl.load_RGI()
#         RGI['Lmax'] = RGI['Lmax'] / 1e3
        rfp = RGI[list(df)[:-1]]

        preds = pd.Series(
            model.predict(rfp.drop('RGIId',axis = 1)).flatten(), name = i
        )
        RGI = pd.concat([RGI,preds], axis = 1)
        RGI.to_pickle(rgi_est_pth)
        print(f'Coregistration {i} predicted RGI')
                # RGI = pd.read_pickle('rgi_est_raw.pkl')

        CenLat      CenLon  Slope  Zmin  Zmed  Zmax     Area  Aspect   Lmax
132  79.885800   95.566700   17.9    31   286   516  171.596      58  11983
309  44.681071   79.739190   13.8  3287  3458  3578    0.635      21   1520
341  43.043653   77.080587   19.1  3427  3764  4185    2.838     359   3131
196  49.741077   87.868423   16.0  2771  3081  3378    2.933      22   3203
246  46.910500    9.248250   20.8  2601  2769  3030    0.666     139   1107
..         ...         ...    ...   ...   ...   ...      ...     ...    ...
80   78.871000   12.161400   13.8   123   388   684    5.016     350   4111
258  46.785900   10.724200   21.2  2742  3232  3576    2.383     238   2474
11   44.164970 -121.784960   17.6  2221  2544  2803    1.170     344   1666
298  44.572747   80.173226   16.6  3447  3719  3997    1.361     203   2396
86   78.978700   14.151800    8.6    85   615  1091  134.647     138  30858

[280 rows x 9 columns]


6766/6766 [==============================] - 5s 758us/step
Coregistration 1 predicted RGI
       CenLat      CenLon  Slope  Zmin  Zmed  Zmax    Area  Aspect   Lmax
26  77.953100   14.964800   15.4   290   498   803   2.884       4   3367
61  49.825649   87.594131   22.4  2643  2987  3853   4.094     326   2892
2   44.164970 -121.784960   17.6  2221  2544  2803   1.170     344   1666
62  49.888519   87.734795   20.1  2994  3385  3799   1.424      22   2597
85  44.857719   79.452026   18.7  3122  3322  3719   1.897      74   2188
..        ...         ...    ...   ...   ...   ...     ...     ...    ...
32  67.915000   18.564000   16.9  1204  1526  1698   1.438      98   1941
92  39.609699   71.569862   10.0  3648  4202  4895  21.345      27   8572
14  78.185100   18.206400   10.2   131   482   694  39.995     232  12142
73  46.906900   12.096500   17.9  2617  2980  3197   1.801      24   1804
19  77.998900   15.030500   10.5   271   454   893   5.136      44   3538

[74 rows x 9 columns]

6766/6766 [==============================] - 5s 680us/step
Coregistration 2 predicted RGI
        CenLat      CenLon  Slope  Zmin  Zmed  Zmax   Area  Aspect  Lmax
18   45.380970 -121.680560   23.9  2011  2426  3310  1.719      42  2786
171  44.890764   79.485387   14.5  3299  3451  3680  1.327      33  1845
107  49.741211   87.886017   26.8  2875  3168  3532  0.791     350  1773
98   49.817616   87.554398   17.2  2661  3079  3387  1.797     336  2701
178  41.786813   77.749273   15.8  3921  4200  4458  0.954      11  2345
..         ...         ...    ...   ...   ...   ...    ...     ...   ...
156  44.572747   80.173226   16.6  3447  3719  3997  1.361     203  2396
91   49.825989   87.506485   20.0  2766  3015  3322  1.704      21  2189
173  44.917077   79.314866   15.9  3202  3348  3570  0.617     345  1631
11   46.896490 -121.789880   20.8  1992  2418  3017  3.029      27  2196
119  47.019900   13.333800   18.2  2525  2905  3296  2.331      71  2310

[141 rows x 9 columns]


6766/6766 [==============================] - 4s 653us/step
Coregistration 3 predicted RGI
        CenLat     CenLon  Slope  Zmin  Zmed  Zmax    Area  Aspect  Lmax
218  44.583903  79.475803   19.1  3258  3497  3751   1.595      20  2160
259 -38.711200 -71.740500   14.4  1783  2167  3059   6.985     237  3688
225  44.885745  79.283745   14.3  3120  3309  3555   2.652      21  2268
111  49.825989  87.506485   20.0  2766  3015  3322   1.704      21  2189
178  46.765300  10.895100   13.7  3064  3353  3485   0.839     195  1099
..         ...        ...    ...   ...   ...   ...     ...     ...   ...
170  46.854800  10.712700   19.5  2631  2931  3475   2.485     358  2558
121  49.888519  87.734795   20.1  2994  3385  3799   1.424      22  2597
140  47.103900  11.028900   19.9  2912  3028  3236   0.280      86   909
214  44.684881  79.725637   13.9  3366  3568  3930   1.528      33  2169
240  39.609699  71.569862   10.0  3648  4202  4895  21.345      27  8572

[191 rows x 9 columns]


6766/6766 [==============================] - 5s 743us/step
Coregistration 4 predicted RGI


In [8]:
import glacierml as gl

In [9]:
# df = gl.coregister_data(4),pth = '/home/simonhans/glacierml/data')
df = gl.coregister_data_reform(data_path,'4')

In [10]:
df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)
df

,RGIId,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Thickness
216,RGI60-01.00037,63.469173,-146.528168,12.0,722,1841,3081,234.583,30,41911,235.0
252,RGI60-01.00570,63.281000,-145.427000,14.0,1162,1858,2438,17.567,172,8639,147.0
33,RGI60-01.01104,58.380000,-134.349000,10.0,676,1123,1494,9.528,327,6332,66.0
217,RGI60-01.16374,61.200000,-140.302000,14.0,2145,2509,2894,5.859,341,3374,66.0
469,RGI60-02.03411,51.177000,-116.330000,15.3,2390,2526,2853,0.872,69,1181,70.0
...,...,...,...,...,...,...,...,...,...,...,...
426,RGI60-17.14088,-33.029600,-70.098400,29.0,2929,5121,5894,8.376,358,8560,59.0
425,RGI60-17.14647,-30.145400,-69.927600,29.3,4664,5187,5525,1.189,148,1596,28.0
424,RGI60-17.15038,-28.538700,-69.731800,16.0,4784,5234,5597,3.140,190,3349,39.0
423,RGI60-17.15087,-28.381400,-69.609100,11.6,5068,5406,5753,5.209,79,3782,67.0
